<img src="img/celery-logo.png">
https://docs.celeryproject.org/

# Celery

Celery - это библитека для создания приложений на основе распределенных очередей сообщений. Она позволяет определять фукнции-обработчики для определенных сообщений в очередях и предоставляет удобный интерфейс для создания таких сообщений и сохранения результатов.

Celery не умеет работать сам по себе - для работы ему требуется сама система очередей. Для этих целей мы будем использовать Redis, так как Redis может работать и как распределенная очередь сообщений (брокер сообщений) и как база данных для хранения результатов. В текущем окружении Redis уже работает и обрабатывает запросы на порту 6379.

Также здесь есть два скрипта для фонового запуска компонентов нашего сервиса. `launch-server.sh` работает также как и в предыдущей лабораторной и запускает Flask приложение. `start-worker.sh` работает похожим образом. Этот скрипт предполагает, что существует скрипт `server.py` и в нем создана переменная `celery_app`, в которой записано основное celery приложение. Если условия удовлетворены, команда в фоновом режиме запускает воркера, который будет следить за очередью.

ВАЖНО - сервер пишет свои логи в `log.txt`, а обработчик celery  - в `log-worker.txt`. Если что-то сломается, то первое куда нужно посмотреть - это в эти файлы. Перезапуск сервера или обработчика также может решить проблему - иногда они просто не успевают перезапустится и поэтому запрос к ним падает с ошибкой.

Попробуем создать вначале просто набор задач и позапускать их в ручном режиме.

In [4]:
! cat $(which launch-server.sh)

#!/bin/bash

set -e

cd /home/jovyan/work

export COMMAND="python3 $1 > log.txt 2>&1"

rm -r __pycache__/ 2> /dev/null || true

(tmux kill-session -t flask-server 2> /dev/null || exit 0) && sleep 3 && tmux new -s flask-server -d "$COMMAND" 2> /dev/null

echo "Success!"

In [5]:
! cat $(which start-worker.sh)

#!/bin/bash

set -e

cd /home/jovyan/work

export COMMAND="celery worker -A server:celery_app -c 2 > log-worker.txt 2>&1"

rm -r __pycache__/ 2> /dev/null || true

(tmux kill-session -t celery-worker 2> /dev/null || exit 0) && sleep 3 && tmux new -s celery-worker -d "$COMMAND" 2> /dev/null

echo "Success!"

In [1]:
%%writefile server.py
from celery import Celery
import time

celery_app = Celery('server', backend='redis://localhost', broker='redis://localhost')  # и брокер и база - redis

@celery_app.task
def add(x, y):
    time.sleep(7.0)
    return x + y

Writing server.py


Теперь запускаем воркера с нашим обработчиком

In [2]:
! start-worker.sh

Success!


Теперь, когда у нас есть работающий воркер, можем импортировать задачи и отправить несколько задачек в очередь.

In [3]:
from server import add

In [4]:
r = add.delay(5, 10)

In [5]:
r

<AsyncResult: c10a2d22-97d4-4858-8c2f-7c377fa66764>

In [6]:
r.id

'c10a2d22-97d4-4858-8c2f-7c377fa66764'

In [7]:
r.ready()

False

In [11]:
r.result

15

In [12]:
r.ready()

True

In [13]:
r.result

15

Отлично! Через некоторое время наша задача посчиталась и мы получили ее результат.

Для того, чтобы получить результат достаточно знать лишь его идентификатор. Попробуем получить результат, зная только идентификатор.

In [14]:
from celery.result import AsyncResult
from server import celery_app

In [15]:
task_id = add.delay(12, 13).id
print(task_id)

ea605124-f752-490b-a79b-42e5125c96ef


In [16]:
r = AsyncResult(task_id, app=celery_app)
print(r.ready())

False


In [20]:
r.result

25

# Подключаем web-сервер

Этих знаний нам должно хватить, чтобы запустить сервис вместе с веб-сервером. Веб-сервер, получая запрос от пользователя будет создавать новую задачу и посылать ее в очередь, возвращая пользователю идентификатор задачи. 

Потом пользователь сможет повторно прийти и узнать у веб-сервера состояние задачи.

In [21]:
%%writefile server.py
from celery import Celery
from celery.result import AsyncResult
import time
from flask import Flask, request
import json


celery_app = Celery('server', backend='redis://localhost', broker='redis://localhost')  # и брокер и база - redis
app = Flask(__name__)  # Основной объект приложения Flask


@celery_app.task
def add(numbers):
    time.sleep(7.0)
    result = 0
    for n in numbers:
        result += n
    return result


@app.route('/sum', methods=["GET", "POST"])
def sum_handler():
    if request.method == 'POST':
        data = request.get_json(force=True)
        numbers = data['numbers']
        
        task = add.delay(numbers) 
            
        response = {
            "task_id": task.id
        }
        return json.dumps(response)
    
    
@app.route('/sum/<task_id>')
def sum_check_handler(task_id):
    task = AsyncResult(task_id, app=celery_app)
    if task.ready():
        response = {
            "status": "DONE",
            "result": task.result
        }
    else:
        response = {
            "status": "IN_PROGRESS"
        }
    return json.dumps(response)


if __name__ == "__main__":
    app.run('0.0.0.0', 8000)

Overwriting server.py


In [22]:
! start-worker.sh

Success!


In [23]:
! launch-server.sh server.py

Success!


In [24]:
import requests

data = {
    "numbers": [2, 34, 65, 23, 79]
}

r = requests.post("http://localhost:8000/sum", json=data)

r_data = r.json()
print(r_data)

{'task_id': 'abdcc70c-f82e-4908-868a-d20ab249bc8c'}


In [25]:
check_url = "http://localhost:8000/sum/{}".format(r_data['task_id'])

r = requests.get(check_url)

print(r.text)

{"status": "IN_PROGRESS"}


Напишем функцию для клиента, который будет ожидать результат

In [26]:
import time

def calc(numbers):
    response = requests.post("http://localhost:8000/sum", json={'numbers': numbers})
    task_id = response.json()['task_id']
    print("Task {}".format(task_id))
    status = "IN_PROGRESS"
    while status != "DONE":
        time.sleep(2.0)
        r = requests.get('http://localhost:8000/sum/{}'.format(task_id))
        status = r.json()['status']
        print('Status - {}'.format(status))
    return r.json()['result']

In [27]:
calc([2, 4, 1, 4, 2, 9])

Task 8d13aee4-84ea-4473-87f3-464debbfb18b
Status - IN_PROGRESS
Status - IN_PROGRESS
Status - IN_PROGRESS
Status - DONE


22

# Сложные модели

Загрузка более сложных моделей практически ничем не отличается от того, что мы делали в предыдущей лабораторной. Попробуем воспроизвести точно такой же сервис, однако теперь используя очередь сообщений.

In [28]:
%%writefile freqmeter.py

import re

class FrequencyMeter:
    def __init__(self):
        self._counter = {}
        self._word_pattern = re.compile(r"[a-z]+")
        
    def fit(self, data):
        for match in self._word_pattern.finditer(data.lower()):
            word = match.group(0)
            if word in self._counter:
                self._counter[word] += 1
            else:
                self._counter[word] = 1
    
    def compute(self, word):
        if word not in self._counter:
            return 0
        return self._counter[word]

Writing freqmeter.py


In [29]:
from freqmeter import FrequencyMeter
import pickle

fmeter = FrequencyMeter()

with open('wizard-of-oz.txt') as f:
    data = f.read()
    
fmeter.fit(data)

raw_data = pickle.dumps(fmeter)

with open('fmeter-model.pickle', 'wb') as f:
    f.write(raw_data)

In [30]:
%%writefile server.py
from celery import Celery
from celery.result import AsyncResult
import time
from flask import Flask, request
import json
import pickle
import re


celery_app = Celery('server', backend='redis://localhost', broker='redis://localhost')  # и брокер и база - redis
app = Flask(__name__)  # Основной объект приложения Flask


def load_model(pickle_path):
    with open(pickle_path, 'rb') as f:
        raw_data = f.read()
        model = pickle.loads(raw_data)
    return model

model = load_model('fmeter-model.pickle')


@celery_app.task
def freq(sentence):
    result = {}
    word_pattern = re.compile(r"[a-z]+")
    for match in word_pattern.finditer(sentence.lower()):
        word = match.group(0)
        result[word] = model.compute(word)
    return result


@app.route('/frequency', methods=["GET", "POST"])
def frequency_handler():
    if request.method == 'POST':
        data = request.get_json(force=True)
        sentence = data['sentence']
        
        task = freq.delay(sentence) 
            
        response = {
            "task_id": task.id
        }
        return json.dumps(response)
    
    
@app.route('/frequency/<task_id>')
def frequency_check_handler(task_id):
    task = AsyncResult(task_id, app=celery_app)
    if task.ready():
        response = {
            "status": "DONE",
            "result": task.result
        }
    else:
        response = {
            "status": "IN_PROGRESS"
        }
    return json.dumps(response)


if __name__ == "__main__":
    app.run('0.0.0.0', 8000)

Overwriting server.py


In [31]:
! start-worker.sh

Success!


In [32]:
! launch-server.sh server.py

Success!


In [33]:
import time

def calc(sentence):
    response = requests.post("http://localhost:8000/frequency", json={'sentence': sentence})
    task_id = response.json()['task_id']
    print("Task {}".format(task_id))
    status = "IN_PROGRESS"
    while status != "DONE":
        time.sleep(2.0)
        r = requests.get('http://localhost:8000/frequency/{}'.format(task_id))
        status = r.json()['status']
        print('Status - {}'.format(status))
    return r.json()['result']

In [37]:
calc("Dorothy lived in the midst of the great Kansas prairies")

Task b6725e4e-8a4b-4eb5-8bc9-acfc9a3f90cd
Status - DONE


{'dorothy': 369,
 'lived': 19,
 'in': 542,
 'the': 3212,
 'midst': 5,
 'of': 974,
 'great': 145,
 'kansas': 51,
 'prairies': 2}